In [1]:
import time
import math

In [36]:
class Node:
    def __init__(self,s,p,a,c):
        self.state = s
        self.parent = p
        self.action = a #the action that was applied to the parent to generate the node
        self.path_cost = c
      
    def __eq__(self, other):
        if (self.path_cost == other.path_cost and self.state == other.state
              and self.parent == other.parent and self.action == other.action):
            return True
        return False
   
    def f(self,problem,h,a):
         return self.path_cost + a*h(problem, self.state[0][0], self.state[1]) #snake's head, seeds
        
    def __lt__(self, other):
        return self.path_cost < other.path_cost

In [3]:
class Problem:
    def __init__(self,seeds,snake,r,c):
        self.seeds = seeds
        self.snake = snake
        self.nrows = r
        self.ncols = c
        self.num_state = 0
        self.unique = 0
        
    def init_state(self):
        s = [self.snake,self.seeds]
        i_state = tuple(tuple(i) for i in s)
        return Node(i_state,None,None,0)
    
    def goal_test(self,state):
        if (len(state[1]) == 1) and state[0][0] == state[1][0]:
            return True
        else:
            return False
        
    def solution(self,node):
        path = []     
        current_node = node
        start_node = self.init_state()
        while current_node != start_node:
            path.insert(0,current_node.action)
            current_node = current_node.parent
    
        return path  
    
    def eat_seed(self,snake,seeds):  
        for i in range(len(seeds)):
            if snake[0] == seeds[i]:
                return i
        return -1
     
    def valid_actions(self,snake,seeds):
     
        left = (snake[0][0], (snake[0][1] - 1)% self.ncols) 
        right = (snake[0][0], (snake[0][1] +1)% self.ncols)
        up = ((snake[0][0] - 1) % self.nrows, snake[0][1])
        down = ((snake[0][0] + 1) % self.nrows, snake[0][1])

        actions = {'U': up,'D':down,'L':left,'R':right}

        for action,result in list(actions.items()):
            if result in snake:
                if snake[-1] != result :
                    actions.pop(action)
                if(snake[-1] == result and (len(snake) == 2)): #not sure
                    actions.pop(action)
                elif snake[-1] == result and self.eat_seed(snake,seeds)!= -1 :#seed
                    actions.pop(action)
            
        return actions
    
    def action_result(self,s,seeds,action):
        snake = list(s)
        sd = list(seeds)
        
        index = self.eat_seed(snake,seeds)
        
        snake.insert(0,self.valid_actions(snake,sd)[action])
        if index == -1:
            snake.pop()
        else:            
            sd.pop(index)
                
        sl = [snake,sd]      
        state = tuple(map(tuple,sl))
        return state

In [4]:
def create_problem(file_path):
    file = open(file_path,'r')
    file_lines = file.readlines()
    lines = []

    for l in file_lines:
        lines.append([int(i) for i in l.strip().split(',')])
    
    nrows, ncols = lines[0]
    snake = [tuple(lines[1])]
    seeds_cnt = lines[2][0]
    seeds = []
    for i in range(seeds_cnt):
        if lines[i+3][2] == 2:
            seeds.append((lines[i+3][0],lines[i+3][1]))
        seeds.append((lines[i+3][0],lines[i+3][1])) 
        
    seeds.sort(key = lambda x: x[0])
    return Problem(seeds,snake,nrows,ncols)


In [5]:
def child_node(problem, parent, action): 
    state = problem.action_result(parent.state[0], parent.state[1], action)
    path_cost = parent.path_cost + 1
    child = Node(state,parent,action,path_cost)
    return child

# BFS

In [6]:
def BFS(problem): 
    
    problem.num_state = 0
    fnode = problem.init_state()
    if (problem.goal_test(fnode.state)):
        return problem.solution(fnode)

    frontier = [fnode] 
    f_set = set() #explored and frontier
    
    while(True):
        if len(frontier) == 0:
           return 'failure'
            
        fnode = frontier[0]
        f_set.add(fnode.state)
        
        frontier.pop(0) 
        
        for action in problem.valid_actions(fnode.state[0],fnode.state[1]):
            problem.num_state +=1
            
            child = child_node(problem, fnode, action)
            if  child.state not in f_set:
                problem.unique+=1
                if problem.goal_test(child.state):
                   return problem.solution(child)
                frontier.append(child)
                f_set.add(child.state)

In [7]:
test3 = create_problem('D:/Term5/هوش/CA/1/test3.txt')

In [8]:
s = time.time()
bfs_sol = BFS(test3)
print(len(bfs_sol), bfs_sol)
e = time.time()
print('time',e - s)
print('All states',test3.num_state)
print('unique states',test3.unique)


25 ['U', 'R', 'D', 'D', 'D', 'R', 'D', 'R', 'R', 'D', 'D', 'R', 'R', 'R', 'U', 'R', 'R', 'D', 'L', 'L', 'L', 'U', 'U', 'L', 'L']
time 5.816001653671265
All states 435666
unique states 194829


# IDS

In [9]:
def DLS(problem, limit):
    src = problem.init_state()
    visited = {str(src.state): src}
    return REC_DLS(src, problem, limit,visited)

def REC_DLS(node, problem, limit,visited):
    
    if problem.goal_test(node.state):
        return problem.solution(node)
    if limit == 0 :
        return 'cutoff'
    else:
        cutoff_occurred = False
        for action in problem.valid_actions(node.state[0],node.state[1]):
            child = child_node(problem, node, action)
            
            child_st = str(child.state)        
            if child_st in visited:
                if (child.path_cost >= visited[child_st].path_cost):
                    continue
                    
            visited[child_st] = child
        
            result = REC_DLS(child, problem, limit-1,visited)
            if result == 'cutoff':
                cutoff_occurred = True
            elif result != 'failure':
                return result
                
        if cutoff_occurred:
            return 'cutoff'
        else:
            return 'failure'

In [10]:
def IDS(problem): 
    depth = 0
    while(True):
        result = DLS(problem, depth)
        if result != 'cutoff':
            return result
        depth +=1
    

In [11]:
s = time.time()
ids_res = IDS(test3)
print(len(ids_res), ids_res)
e = time.time()
print('time',e - s)

25 ['U', 'R', 'D', 'D', 'D', 'R', 'D', 'R', 'R', 'D', 'D', 'R', 'R', 'R', 'U', 'R', 'R', 'D', 'L', 'L', 'L', 'U', 'U', 'L', 'L']
time 47.118069648742676


# A*

In [15]:
def nearest_seed(problem, coordinate, seeds):
    min_dist = float('inf')
    for s in seeds:
        y1 = s[0] - coordinate[0]
        x1 =  s[1] - coordinate[1]
        dist = min(y1 % problem.nrows,-y1 % problem.nrows) + min(x1 % problem.ncols,-x1 % problem.ncols)
        if dist < min_dist:
            min_dist = dist
    if len(seeds) == 0:
        min_dist = 0     
    return min_dist

def furthest_seed(problem, coordinate, seeds):
    max_dist = 0
    for s in seeds:
        y1 = s[0] - coordinate[0]
        x1 =  s[1] - coordinate[1]
        dist = min(y1 % problem.nrows,-y1 % problem.nrows) + min(x1 % problem.ncols,-x1 % problem.ncols)
        if dist > max_dist:
            max_dist = dist
    if len(seeds) == 0:
        max_dist = 0
        
    return max_dist

In [48]:
def Astar(problem,h,a):
    
   problem.num_state = 0
   problem.unique = 0
   
   src = problem.init_state()
   frontier = [src]
   fdict = {str(src.state):src}  #frontier 
   explored = set()
   
   while len(frontier) > 0:
    
      frontier.sort(key = lambda x:x.f(problem,h,a))  #choose node with lowest f
      fnode = frontier[0]

      if problem.goal_test(fnode.state):
         return problem.solution(fnode)
   
      frontier.pop(0)
      del fdict[str(fnode.state)]
      explored.add(fnode.state)

      for action in problem.valid_actions(fnode.state[0],fnode.state[1]):
         problem.num_state +=1
      
         child = child_node(problem, fnode, action)
         if child.state in explored:
            continue
         child_st = str(child.state)

         if child_st in fdict:       
            if fdict[child_st].path_cost > child.path_cost:
            
               fdict[child_st].path_cost = child.path_cost
               fdict[child_st].parent =  child.parent
               fdict[child_st].action = child.action
         else:
            problem.unique +=1  
            frontier.append(child)
            fdict[child_st] = child

In [49]:
problem = create_problem('D:/Term5/هوش/CA/1/test1.txt')

In [50]:
s = time.time()
a = 1
astar_sol = Astar(problem,nearest_seed,a)
print(len(astar_sol),astar_sol)
e = time.time()
print('time',e - s)
print('All states',problem.num_state)
print('unique states',problem.unique)

12 ['D', 'L', 'U', 'L', 'D', 'R', 'U', 'U', 'L', 'U', 'L', 'L']
time 5.732997417449951
All states 6910
unique states 3894


In [51]:
s = time.time()
a = 1
astar_sol = Astar(problem,furthest_seed,a)
print(len(astar_sol),astar_sol)
e = time.time()
print('time',e - s)
print('All states',problem.num_state)
print('unique states',problem.unique)

12 ['D', 'L', 'D', 'D', 'R', 'R', 'R', 'R', 'D', 'D', 'D', 'R']
time 2.706003189086914
All states 4560
unique states 2633
